In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import scipy.optimize as opt
from sklearn.metrics import classification_report  # 这个包是评价报告

def load_mat(path):
    data=loadmat('ex4data1.mat')
    X=data['X']
    y=data['y'].flatten()
    return X,y

#随机画100个数字
def plot_100_images(X):
    #在5000个数字中随机选取100个
    index=np.random.choice(range(5000),100)
    images=X[index]#100个照片
    fig,ax_array=plt.subplots(10,10,sharey=True,sharex=True,figsize=(8,8))
    for r in range(10):#r代表行
        for c in range(10):#c代表列
            ax_array[r,c].matshow(images[r*10+c].reshape(20,20),cmap='gray_r')
    plt.xticks([])
    plt.yticks([])
    plt.show()

X,y=load_mat('ex4data1.mat')
plot_100_images(X)

from sklearn.preprocessing import OneHotEncoder
def expand_y(y):
    result=[]
    for i in y:
        y_array=np.zeros(10)
        y_array[i-1]=1
        result.append(y_array)
    return np.array(result)

#例如y[0]=6转化为y[0]=[0,0,0,0,0,1,0,0,0,0]。

raw_X, raw_y = load_mat('ex4data1.mat')
#在第0列插入1列
X = np.insert(raw_X, 0, 1, axis=1)
y = expand_y(raw_y)
X.shape, y.shape
'''
((5000, 401), (5000, 10))
'''

#读取权重
def load_weight(path):
    data=loadmat(path)
    return data['Theta1'],data['Theta2']
t1, t2 = load_weight('ex4weights.mat')
t1.shape, t2.shape
# ((25, 401), (10, 26))

def serialize(a, b):
    '''展开参数'''
    return np.r_[a.flatten(),b.flatten()]

theta = serialize(t1, t2)  # 扁平化参数，25*401+10*26=10285
theta.shape  # (10285,)

def deserialize(seq):
    '''提取参数'''
    return seq[:25*401].reshape(25, 401), seq[25*401:].reshape(10, 26)

#前馈和代价函数
#确保每层的单元数，注意输出时加一个偏置单元，s(1)=400+1，s(2)=25+1，s(3)=10。
def sigmoid(z):
    return 1/(1+np.exp(-z))
def feed_forward(theta,X):
    t1,t2=deserialize(theta)#t1(25,401)  t2(10,26)
    #X(5000,401)
    a1=X
    z2=a1@t1.T#(5000,25)
    a2=np.insert(sigmoid(z2),0,1,axis=1)#(5000,26)
    z3=a2@t2.T#(5000,10)
    a3=sigmoid(z3)#(5000,10)
    return a1,z2,a2,z3,a3



#cost function
#不带正则化
def cost(theta, X, y):
    a1, z2, a2, z3, h = feed_forward(theta, X)
    J = 0
    for i in range(len(X)):
        first = - y[i] * np.log(h[i])
        second = (1 - y[i]) * np.log(1 - h[i])
        J = J + np.sum(first - second)
    J = J / len(X)
    return J
'''
     # or just use verctorization
     J = - y * np.log(h) - (1 - y) * np.log(1 - h)
     return J.sum() / len(X)
'''
cost(theta, X, y)  # 0.2876291651613189
#一次正向传播的损失

#正则化代价函数
def regularized_cost(theta, X, y, l=1):
    '''正则化时忽略每层的偏置项，也就是参数矩阵的第一列'''
    t1, t2 = deserialize(theta)
    reg = np.sum(t1[:,1:] ** 2) + np.sum(t2[:,1:] ** 2)  # or use np.power(a, 2)
    return l / (2 * len(X)) * reg + cost(theta, X, y)

regularized_cost(theta, X, y, 1)  # 0.38376985909092354

#反向传播
def sigmoid_gradient(z):
    return sigmoid(z) * (1 - sigmoid(z))

def random_init(size):
    '''从服从的均匀分布的范围中随机返回size大小的值'''
    return np.random.uniform(-0.12, 0.12, size)
def gradient(theta, X, y):
    '''
    unregularized gradient, notice no d1 since the input layer has no error 
    return 所有参数theta的梯度，故梯度D(i)和参数theta(i)同shape，重要。
    '''
    t1, t2 = deserialize(theta)
    a1, z2, a2, z3, h = feed_forward(theta, X)
    d3 = h - y # (5000, 10)
    d2 = d3 @ t2[:,1:] * sigmoid_gradient(z2)  # (5000, 25)
    D2 = d3.T @ a2  # (10, 26)
    D1 = d2.T @ a1 # (25, 401)
    D = (1 / len(X)) * serialize(D1, D2)  # (10285,)
    
    return D

def gradient_checking(theta, X, y, e):
    def a_numeric_grad(plus, minus):
        """
        对每个参数theta_i计算数值梯度，即理论梯度。
        """
        return (regularized_cost(plus, X, y) - regularized_cost(minus, X, y)) / (e * 2)
   
    numeric_grad = [] 
    for i in range(len(theta)):
        plus = theta.copy()  # deep copy otherwise you will change the raw theta
        minus = theta.copy()
        plus[i] = plus[i] + e
        minus[i] = minus[i] - e
        grad_i = a_numeric_grad(plus, minus)
        numeric_grad.append(grad_i)
    
    numeric_grad = np.array(numeric_grad)
    analytic_grad = regularized_gradient(theta, X, y)
    diff = np.linalg.norm(numeric_grad - analytic_grad) / np.linalg.norm(numeric_grad + analytic_grad)

    print('If your backpropagation implementation is correct,\nthe relative difference will be smaller than 10e-9 (assume epsilon=0.0001).\nRelative Difference: {}\n'.format(diff))


def regularized_gradient(theta, X, y, l=1):
    """不惩罚偏置单元的参数"""
    a1, z2, a2, z3, h = feed_forward(theta, X)
    D1, D2 = deserialize(gradient(theta, X, y))
    t1[:,0] = 0
    t2[:,0] = 0
    reg_D1 = D1 + (l / len(X)) * t1
    reg_D2 = D2 + (l / len(X)) * t2
    
    return serialize(reg_D1, reg_D2)

def nn_training(X, y):
    init_theta = random_init(10285)  # 25*401 + 10*26

    res = opt.minimize(fun=regularized_cost,
                       x0=init_theta,
                       args=(X, y, 1),
                       method='TNC',
                       jac=regularized_gradient,
                       options={'maxiter': 400})
    return res

def accuracy(theta, X, y):
    _, _, _, _, h = feed_forward(res.x, X)
    y_pred = np.argmax(h, axis=1) + 1
    print(classification_report(y, y_pred))

def plot_hidden(theta):
    t1, _ = deserialize(theta)
    t1 = t1[:, 1:]
    fig,ax_array = plt.subplots(5, 5, sharex=True, sharey=True, figsize=(6,6))
    for r in range(5):
        for c in range(5):
            ax_array[r, c].matshow(t1[r * 5 + c].reshape(20, 20), cmap='gray_r')
            plt.xticks([])
            plt.yticks([])
    plt.show()


SyntaxError: unexpected EOF while parsing (<ipython-input-2-d5d4fc1ae0c2>, line 20)